### THE GENERATED CODE BY LLM

In [1]:
from z3 import *

def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    
    參數:
      big_category: 字串, "cement" 或 "oilgas"
      sub_category: 字串
         若 big_category == "cement":
           "white"         -> 白水泥或有色水泥 (600 NT$/ton)  (貨物稅條例 第7條)
           "bute1"         -> 卜特蘭一型 (320 NT$/ton)         (貨物稅條例 第7條)
           "bute2"         -> 卜特蘭高爐 (196 NT$/ton)         (原法條為280元，經調整)
           "other_cement"  -> 代水泥及其他水泥 (440 NT$/ton)    (貨物稅條例 第7條)
         若 big_category == "oilgas":
           "gasoline"      -> 汽油 (6830 NT$/unit)             (貨物稅條例 第10條)
           "diesel"        -> 柴油 (3990 NT$/unit)             (貨物稅條例 第10條)
           "kerosene"      -> 煤油 (4250 NT$/unit)             (貨物稅條例 第10條)
           "aviation"      -> 航空燃油 (610 NT$/unit)          (貨物稅條例 第10條)
           "fuel_oil"      -> 燃料油 (110 NT$/unit)            (貨物稅條例 第10條)
           "solvent_oil"   -> 溶劑油 (720 NT$/unit)            (貨物稅條例 第10條)
           "lpg"           -> 液化石油氣 (690 NT$/ton)         (貨物稅條例 第10條)
      quantity: 當月份出廠應稅數量 (整數)
      
    計算公式:
      應納稅額 = (單位稅額) * (數量)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大種類決定 mapping，mapping 中之數值依據各法條規定
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元 (貨物稅條例 第7條)
            "bute1": 320,          # 卜特蘭一型，每公噸320元 (貨物稅條例 第7條)
            "bute2": 196,          # 卜特蘭高爐，每公噸196元 (原法條為280元，經調整)
            "other_cement": 440    # 代水泥及其他水泥，每公噸440元 (貨物稅條例 第7條)
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元 (貨物稅條例 第10條)
            "diesel": 3990,        # 柴油，每公秉3990元 (貨物稅條例 第10條)
            "kerosene": 4250,      # 煤油，每公秉4250元 (貨物稅條例 第10條)
            "aviation": 610,       # 航空燃油，每公秉610元 (貨物稅條例 第10條)
            "fuel_oil": 110,       # 燃料油，每公秉110元 (貨物稅條例 第10條)
            "solvent_oil": 720,    # 溶劑油，每公秉720元 (貨物稅條例 第10條)
            "lpg": 690             # 液化石油氣，每公噸690元 (貨物稅條例 第10條)
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    # 加入 constraint: tax_amount = unit_tax * quantity
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    
    參數:
      big_category: 字串，必須為下列之一：
           "rubber"    (橡膠輪胎)
           "beverage"  (飲料品)
           "flat_glass" (平板玻璃)
           "electrical" (電器)
           "vehicle"   (車輛)
      sub_category: 字串，依據大類別定義：
         若 big_category == "rubber":
           "bus_truck" -> 大客車、大貨車使用者 (從價徵收10%)
           "other"     -> 其他 (從價徵收15%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "beverage":
           "diluted_juice" -> 稀釋天然果蔬汁 (從價徵收8%)
           "other"         -> 其他飲料品 (從價徵收15%)
           "exempt"        -> 免稅 (0%)
         若 big_category == "flat_glass":
           "regular"   -> 一般 (從價徵收10%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "electrical":
           "refrigerator"           -> 電冰箱 (從價徵收13%)
           "color_tv"               -> 彩色電視機 (從價徵收13%)
           "central_aircon"         -> 冷暖氣機(中央系統型) (從價徵收15%)
           "non_central_aircon"     -> 冷暖氣機(非中央系統型) (從價徵收20%)
           "dehumidifier"           -> 除濕機 (從價徵收15%)
           "videorecorder"          -> 錄影機 (從價徵收13%)
           "non_portable_recordplayer" -> 電唱機(非手提32公分) (從價徵收10%)
           "portable_recordplayer"  -> 電唱機(手提32公分以下) (免稅 0%)
           "tape_recorder"          -> 錄音機 (從價徵收10%)
           "stereo"                 -> 音響組合 (從價徵收10%)
           "electric_oven"          -> 電烤箱 (從價徵收15%)
         若 big_category == "vehicle":
           "compact_car"   -> 小客車(2000cc以下) (從價徵收25%)
           "large_car"     -> 小客車(2000cc以上) (從價徵收30%)
           "truck_bus_others" -> 貨車、大客車及其他 (從價徵收15%)
           "motorcycle"    -> 機車 (從價徵收17%)
      quantity: 當月份出廠應稅數量 (整數)
      assessed_price: 每單位完稅價格 (例如由銷售價格/(1+稅率) 得到)
      is_electric: (僅針對 vehicle 類) 布林值，若為 True 則稅率依規定減半
      
    計算公式:
      應納稅額 = assessed_price * (從價稅率) * quantity
      (若為車輛且 is_electric 為 True，則稅率折半)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大類別與子類別決定稅率
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        # 若為電動車或油電混合車輛，則稅率折半
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    # 加入 constraint: tax_amount = assessed_price * rate * quantity
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# 測試範例
if __name__ == '__main__':
    # 範例1：水泥類，選擇白水泥，出廠數量10公噸
    tax1 = compute_tax_fixed("cement", "white", 10)
    print("水泥類 Tax:", tax1)  # 預期 600 * 10 = 6000
    
    # 範例2：油氣類，選擇汽油，出廠數量5 (單位：公秉)
    tax2 = compute_tax_fixed("oilgas", "gasoline", 5)
    print("油氣類 Tax:", tax2)  # 預期 6830 * 5 = 34150
    
    # 範例3：其他類目 - 電器，選擇冷暖氣機(中央系統型)，完稅價格2000元，出廠數量2台
    tax3 = compute_tax_others("electrical", "central_aircon", 2, 2000)
    print("電器 Tax:", tax3)  # 預期 2000 * 0.15 * 2 = 600
    
    # 範例4：其他類目 - 車輛，選擇小客車(2000cc以下) (25%)，非電動車，完稅價格800000元，出廠數量3輛
    tax4 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=False)
    print("車輛 Tax:", tax4)  # 預期 800000 * 0.25 * 3 = 600000
    
    # 範例5：其他類目 - 車輛，同上但若為則稅率減半 (0.25/2 = 0.125)
    tax5 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=True)
    print("電動車 Tax:", tax5)  # 預期 800000 * 0.125 * 3 = 300000


水泥類 Tax: 6000
油氣類 Tax: 34150
電器 Tax: 600
車輛 Tax: 600000
電動車 Tax: 300000


### Selenium

In [7]:
import random
import time
import re
from decimal import Decimal, ROUND_DOWN

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    StaleElementReferenceException,
    TimeoutException,
)

URL = "https://www.etax.nat.gov.tw/etwmain/etw158w/79"


# ------------------------------
# 計算（Decimal）
# ------------------------------
def compute_tax_fixed(unit_tax: int, quantity: int) -> Decimal:
    return Decimal(unit_tax) * Decimal(quantity)

def compute_tax_others(quantity: int, assessed_price: int, rate_percent: Decimal) -> Decimal:
    return Decimal(assessed_price) * (rate_percent / Decimal(100)) * Decimal(quantity)

def trunc_decimal_to_int(x: Decimal) -> Decimal:
    # 規則：local tax 結果直接取整數位（截斷小數位）
    return x.quantize(Decimal("1"), rounding=ROUND_DOWN)


# ------------------------------
# 逐字輸入（跟你原本一樣）
# ------------------------------
def human_type(element, text: str, delay=0.1):
    for ch in text:
        element.send_keys(ch)
        time.sleep(delay)
    element.send_keys(Keys.TAB)

def safe_clear(driver, element):
    try:
        element.clear()
    except Exception:
        driver.execute_script("arguments[0].value = '';", element)

def is_editable(el) -> bool:
    ro = el.get_attribute("readonly")
    dis = el.get_attribute("disabled")
    return (not ro) and (not dis) and el.is_enabled()


# ------------------------------
# 解析數字
# ------------------------------
def parse_decimal(text: str) -> Decimal | None:
    if text is None:
        return None
    s = text.strip().replace(",", "")
    if not s:
        return None
    m = re.search(r"-?\d+(?:\.\d+)?", s)
    return Decimal(m.group(0)) if m else None


# ------------------------------
# 依照 input 限制產生合法整數
# ------------------------------
def get_int_constraints(el, default_min=1, default_max=50):
    min_attr = el.get_attribute("min")
    max_attr = el.get_attribute("max")
    maxlength = el.get_attribute("maxlength")

    min_v = int(min_attr) if (min_attr and re.fullmatch(r"\d+", min_attr)) else default_min
    max_v = int(max_attr) if (max_attr and re.fullmatch(r"\d+", max_attr)) else default_max

    if maxlength and re.fullmatch(r"\d+", maxlength):
        ml = int(maxlength)
        max_by_len = (10 ** ml) - 1
        max_v = min(max_v, max_by_len)

    if max_v < min_v:
        max_v = min_v

    # 保守：避免打太長造成輸入很慢
    max_v = min(max_v, 9_999_999)
    return min_v, max_v


def pick_valid_option_values(sel: Select):
    out = []
    for opt in sel.options:
        v = (opt.get_attribute("value") or "").strip()
        t = (opt.text or "").strip()
        if not opt.is_enabled():
            continue
        if not v or v == "0":
            continue
        if "請選擇" in t:
            continue
        out.append((v, t))
    return out


# ------------------------------
# overlay 防呆
# ------------------------------
def dismiss_gsc_overlay(driver):
    try:
        close_btns = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-close-btn")
        for b in close_btns:
            if b.is_displayed() and b.is_enabled():
                try:
                    b.click()
                    time.sleep(0.1)
                except Exception:
                    pass

        try:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
            time.sleep(0.05)
        except Exception:
            pass

        overlays = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-wrapper-overlay")
        for ov in overlays:
            if ov.is_displayed():
                driver.execute_script(
                    "arguments[0].style.display='none'; arguments[0].style.pointerEvents='none';",
                    ov
                )
    except Exception:
        pass


def click_calc_near_anchor(driver, anchor_id: str, fallback_index: int):
    dismiss_gsc_overlay(driver)

    anchor = driver.find_element(By.ID, anchor_id)
    buttons = [
        b for b in driver.find_elements(By.XPATH, "//button[@title='計算']")
        if b.is_displayed() and b.is_enabled()
    ]
    if not buttons:
        raise RuntimeError("頁面上找不到可點擊的「計算」按鈕")

    try:
        target = driver.execute_script(
            """
            const anchor = arguments[0];
            const btns = arguments[1];
            const ay = anchor.getBoundingClientRect().top;
            let best = null, bestd = 1e18;
            for (const b of btns) {
              const r = b.getBoundingClientRect();
              const d = Math.abs(r.top - ay);
              if (d < bestd) { bestd = d; best = b; }
            }
            return best;
            """,
            anchor, buttons
        )
    except Exception:
        target = buttons[fallback_index] if len(buttons) > fallback_index else buttons[0]

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", target)
    time.sleep(0.15)

    for _ in range(3):
        try:
            dismiss_gsc_overlay(driver)
            target.click()
            return
        except ElementClickInterceptedException:
            dismiss_gsc_overlay(driver)
            try:
                driver.execute_script("arguments[0].click();", target)
                return
            except Exception:
                time.sleep(0.2)
        except StaleElementReferenceException:
            time.sleep(0.2)
            return click_calc_near_anchor(driver, anchor_id, fallback_index)

    raise RuntimeError("點擊「計算」按鈕失敗（可能仍被 overlay / modal 擋住）")


# ------------------------------
# sub UI 偵測：radio 或 select4
# ------------------------------
def detect_sub_ui(driver, timeout=10.0) -> str | None:
    end = time.time() + timeout
    while time.time() < end:
        # radio（車輛）
        radios = [r for r in driver.find_elements(By.NAME, "radio1") if r.is_displayed() and r.is_enabled()]
        if radios:
            return "radio"

        # select4（其他）
        try:
            el = driver.find_element(By.ID, "select4")
            if el.is_displayed() and el.is_enabled():
                sel4 = Select(el)
                if len(pick_valid_option_values(sel4)) > 0:
                    return "select4"
        except Exception:
            pass

        time.sleep(0.1)
    return None


def choose_select3_with_valid_sub_ui(driver, wait: WebDriverWait, *,
                                    timeout_per_try=6.0, max_tries=20) -> tuple[str, str, str]:
    """
    目標：避免「找不到 sub UI」=> 不略過
    做法：從 select3 的可用 option 裡亂數挑選/輪詢，直到某個 option 真的產生 radio 或 select4
    回傳: (value, text, ui_type)
    """
    select3 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select3"))))
    opts = pick_valid_option_values(select3)
    if not opts:
        raise RuntimeError("select3 沒有可選項目（頁面可能未正確載入）")

    random.shuffle(opts)
    tries = 0

    # 允許重複抽（因為有些選項可能偶發載入慢）
    while tries < max_tries:
        v, t = opts[tries % len(opts)]
        tries += 1

        select3.select_by_value(v)
        time.sleep(0.5)
        dismiss_gsc_overlay(driver)

        # 有時需要滾到從價稅區塊，避免 lazy render（保守做法）
        try:
            paid = driver.find_element(By.ID, "paid")
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", paid)
        except Exception:
            pass

        ui = detect_sub_ui(driver, timeout=timeout_per_try)
        if ui in ("radio", "select4"):
            return v, t, ui

    # 全部試過仍不行：把 select3 的選項印出來方便你核對網站是不是改版
    debug_list = "\n".join([f"- value={vv}, text={tt}" for vv, tt in opts])
    raise RuntimeError(
        "select3 的選項中找不到會產生 radio/select4 的類別（網站可能改版或 UI 改成別的元件）\n"
        f"select3 options:\n{debug_list}"
    )


# ------------------------------
# 讀取結果：找離 anchor 最近的 span.text-danger
# ------------------------------
def read_result_near_anchor(driver, anchor_id: str, timeout: float = 10.0, fallback_index: int = 0) -> Decimal | None:
    end = time.time() + timeout
    while time.time() < end:
        spans = [s for s in driver.find_elements(By.CSS_SELECTOR, "span.text-danger") if s.is_displayed()]
        candidates = []
        for s in spans:
            d = parse_decimal(s.text)
            if d is not None:
                candidates.append((s, d))

        if candidates:
            try:
                anchor = driver.find_element(By.ID, anchor_id)
                idx = driver.execute_script(
                    """
                    const anchor = arguments[0];
                    const spans = arguments[1];
                    const ay = anchor.getBoundingClientRect().top;
                    let bestIdx = 0, bestD = 1e18;
                    for (let i=0; i<spans.length; i++) {
                      const r = spans[i].getBoundingClientRect();
                      const d = Math.abs(r.top - ay);
                      if (d < bestD) { bestD = d; bestIdx = i; }
                    }
                    return bestIdx;
                    """,
                    anchor, [c[0] for c in candidates]
                )
                return candidates[int(idx)][1]
            except Exception:
                idx = min(fallback_index, len(candidates) - 1)
                return candidates[idx][1]

        time.sleep(0.1)

    return None


# ------------------------------
# 主程式
# ------------------------------
def run_tax_calculator_test(total_cases=100, seed=42):
    random.seed(seed)

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 15)

    driver.delete_all_cookies()
    driver.get(URL)
    time.sleep(2)
    dismiss_gsc_overlay(driver)

    # 確保核心元件存在（避免一開始就跑）
    wait.until(EC.presence_of_element_located((By.ID, "select1")))
    wait.until(EC.presence_of_element_located((By.ID, "select3")))

    mismatches = 0

    for i in range(total_cases):
        print(f"\n===== 測試案例 {i+1}/{total_cases} =====")
        tax_type = random.choice([0, 1])  # 0 固定稅額 / 1 從價稅

        try:
            if tax_type == 0:
                # ------------------------------
                # 固定稅額：select1 + select2 + taxCount1
                # ------------------------------
                select1 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select1"))))
                big_opts = pick_valid_option_values(select1)
                big_v, big_t = random.choice(big_opts)
                select1.select_by_value(big_v)
                time.sleep(0.5)

                wait.until(lambda d: len(pick_valid_option_values(Select(d.find_element(By.ID, "select2")))) > 0)
                select2 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select2"))))
                sub_opts = pick_valid_option_values(select2)
                sub_v, sub_t = random.choice(sub_opts)
                select2.select_by_value(sub_v)
                time.sleep(0.5)

                # select2 的 value 通常就是單位稅額（數字）
                unit_tax = int(Decimal(sub_v))

                # tax 欄位如果可編輯就填（跟你原本一致）
                tax_input = wait.until(EC.presence_of_element_located((By.ID, "tax")))
                if tax_input.is_displayed() and is_editable(tax_input):
                    safe_clear(driver, tax_input)
                    human_type(tax_input, str(unit_tax), delay=0.1)
                    time.sleep(0.5)

                taxCount1 = wait.until(EC.element_to_be_clickable((By.ID, "taxCount1")))
                qmin, qmax = get_int_constraints(taxCount1, default_min=1, default_max=50)
                quantity = random.randint(qmin, qmax)

                safe_clear(driver, taxCount1)
                human_type(taxCount1, str(quantity), delay=0.1)
                time.sleep(0.5)

                local_raw = compute_tax_fixed(unit_tax, quantity)
                local = trunc_decimal_to_int(local_raw)  # local 規則：截斷

                click_calc_near_anchor(driver, anchor_id="taxCount1", fallback_index=0)
                time.sleep(1.2)

                online = read_result_near_anchor(driver, anchor_id="taxCount1", timeout=10.0, fallback_index=0)

                if online is None or online != local:
                    print("不一致測試案例 (固定稅額):")
                    print(f"  select1: {big_t}({big_v}), select2: {sub_t}({sub_v})")
                    print(f"  unit_tax={unit_tax}, quantity={quantity}")
                    print(f"  本地 raw={local_raw} -> trunc={local} | 網頁={online}")
                    mismatches += 1

            else:
                # ------------------------------
                # 從價稅：select3 + (radio 或 select4) + paid + taxCount2
                # ------------------------------
                big_v, big_t, ui = choose_select3_with_valid_sub_ui(driver, wait, timeout_per_try=6.0, max_tries=25)
                print(f"[從價稅] select3: {big_t}({big_v}) => ui={ui}")
                time.sleep(0.2)

                is_electric = False
                rate_percent = Decimal(0)

                if ui == "radio":
                    radios = [r for r in driver.find_elements(By.NAME, "radio1") if r.is_displayed() and r.is_enabled()]
                    chosen = random.choice(radios)
                    chosen.click()
                    time.sleep(0.5)
                    rate_percent = Decimal(chosen.get_attribute("value"))

                    # 電動/油電 checkbox（若存在）
                    try:
                        checkbox = driver.find_element(By.ID, "carCheckBox")
                        if checkbox.is_displayed() and checkbox.is_enabled():
                            is_electric = random.choice([True, False])
                            if is_electric and not checkbox.is_selected():
                                checkbox.click()
                            elif (not is_electric) and checkbox.is_selected():
                                checkbox.click()
                            time.sleep(0.5)

                            if is_electric:
                                rate_percent = rate_percent / Decimal(2)
                    except Exception:
                        pass

                else:
                    wait.until(lambda d: len(pick_valid_option_values(Select(d.find_element(By.ID, "select4")))) > 0)
                    select4 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select4"))))
                    sub_opts = pick_valid_option_values(select4)
                    sub_v, sub_t = random.choice(sub_opts)
                    select4.select_by_value(sub_v)
                    time.sleep(0.5)

                    rate_percent = Decimal(sub_v)

                    # taxRate 如果可編輯就填（跟你原本一致）
                    taxRate_input = wait.until(EC.presence_of_element_located((By.ID, "taxRate")))
                    if taxRate_input.is_displayed() and is_editable(taxRate_input):
                        safe_clear(driver, taxRate_input)
                        human_type(taxRate_input, str(int(rate_percent)), delay=0.1)
                        time.sleep(0.5)

                paid_input = wait.until(EC.element_to_be_clickable((By.ID, "paid")))
                pmin, pmax = get_int_constraints(paid_input, default_min=1, default_max=1_000_000)
                assessed_price = random.randint(pmin, pmax)
                safe_clear(driver, paid_input)
                human_type(paid_input, str(assessed_price), delay=0.1)
                time.sleep(0.5)

                taxCount2 = wait.until(EC.element_to_be_clickable((By.ID, "taxCount2")))
                qmin, qmax = get_int_constraints(taxCount2, default_min=1, default_max=50)
                quantity = random.randint(qmin, qmax)
                safe_clear(driver, taxCount2)
                human_type(taxCount2, str(quantity), delay=0.1)
                time.sleep(0.5)

                local_raw = compute_tax_others(quantity, assessed_price, rate_percent)
                local = trunc_decimal_to_int(local_raw)  # local 規則：截斷

                click_calc_near_anchor(driver, anchor_id="taxCount2", fallback_index=1)
                time.sleep(1.2)

                online = read_result_near_anchor(driver, anchor_id="taxCount2", timeout=10.0, fallback_index=1)

                if online is None or online != local:
                    print("不一致測試案例 (從價稅):")
                    print(f"  select3: {big_t}({big_v}) | ui={ui} | rate%={rate_percent} | is_electric={is_electric}")
                    print(f"  paid={assessed_price}, quantity={quantity}")
                    print(f"  本地 raw={local_raw} -> trunc={local} | 網頁={online}")
                    mismatches += 1

        except Exception as e:
            print(f"[案例執行錯誤] {type(e).__name__}: {e}")
            mismatches += 1

        # 重置
        driver.refresh()
        time.sleep(2)
        dismiss_gsc_overlay(driver)
        wait.until(EC.presence_of_element_located((By.ID, "select3")))

    print(f"\n總測試次數: {total_cases}, 不一致次數: {mismatches}")
    driver.quit()


if __name__ == "__main__":
    run_tax_calculator_test(total_cases=100, seed=42)



===== 測試案例 1/100 =====

===== 測試案例 2/100 =====

===== 測試案例 3/100 =====

===== 測試案例 4/100 =====

===== 測試案例 5/100 =====

===== 測試案例 6/100 =====
[從價稅] select3: 飲料品(2) => ui=select4

===== 測試案例 7/100 =====

===== 測試案例 8/100 =====

===== 測試案例 9/100 =====
[從價稅] select3: 電器(4) => ui=select4

===== 測試案例 10/100 =====

===== 測試案例 11/100 =====
[從價稅] select3: 電器(4) => ui=select4

===== 測試案例 12/100 =====
[從價稅] select3: 平板玻璃(3) => ui=select4

===== 測試案例 13/100 =====
[從價稅] select3: 電器(4) => ui=select4

===== 測試案例 14/100 =====

===== 測試案例 15/100 =====
[從價稅] select3: 飲料品(2) => ui=select4

===== 測試案例 16/100 =====

===== 測試案例 17/100 =====
[從價稅] select3: 飲料品(2) => ui=select4

===== 測試案例 18/100 =====
[從價稅] select3: 平板玻璃(3) => ui=select4

===== 測試案例 19/100 =====
[從價稅] select3: 橡膠輪軩(1) => ui=select4

===== 測試案例 20/100 =====
[從價稅] select3: 飲料品(2) => ui=select4

===== 測試案例 21/100 =====
[從價稅] select3: 電器(4) => ui=select4

===== 測試案例 22/100 =====
[從價稅] select3: 飲料品(2) => ui=select4

===== 測試案例 23/100 =====
[從價

### According to the selenium test above, we have 0 mismatches against the MoF protal